In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"

#!import ./config/Settings.cs

using Microsoft.SemanticKernel;
using System.IO;

var kernelBuilder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Grab the locally stored credentials from the settings.json file. 
// Name the service as "davinci" — assuming that you're using one of the davinci completion models. 
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernelBuilder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    kernelBuilder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = kernelBuilder.Build();

In [ ]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory

var skillDT = kernel.ImportPluginFromPromptDirectory(Path.Combine(skillsDirectory, "DesignThinkingSkill"));

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;

// The default input variable 
var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

var empathyResult = await kernel.InvokeAsync(skillDT["Empathize"], new(){ ["input"] = input });

Console.WriteLine(empathyResult);

In [ ]:
var defineResult = await kernel.InvokeAsync(skillDT["Define"], new(){ ["input"] = empathyResult.ToString() });

Console.WriteLine(defineResult);

In [ ]:
var ideateResult = await kernel.InvokeAsync(skillDT["Ideate"], new(){ ["input"] = defineResult.ToString() });

Console.WriteLine(ideateResult.ToString());

In [ ]:
#r "nuget: Markdig.Signed, 0.34.0"
using Microsoft.AspNetCore.Html;
using Markdig;
using System.Text.Json;
using System.Text.Json.Serialization;

var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

var myResult = await kernel.InvokeAsync(skillDT["Empathize"], /*skillDT["Define"], skillDT["Ideate"],*/ new(){ ["input"] = input });

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml("# Ideate: 'Design Thinking' directions generated from customer empathy (Empathize) and problem definition (Define)\n\n" + myResult.ToString(), myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent